In [125]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def svd_truncation(matrix, rank):
    U, s, Vt = np.linalg.svd(matrix)

    U_truncated = U[:,:rank]
    Vt_truncated = Vt[:rank]
    s_truncated = s[:rank]

    return np.shape(matrix), U_truncated, Vt_truncated, s_truncated

In [ ]:
def matrix_padding(shape, U_truncated, Vt_truncated, s_truncated):
    n, m = shape

    U_padded = np.pad(U_truncated, ((0, n - U_truncated.shape[0]), (0, n - U_truncated.shape[1])), mode='constant')

    Vt_padded = np.pad(Vt_truncated, ((0, m - Vt_truncated.shape[0]), (0, m - Vt_truncated.shape[1])), mode='constant')

    Z_padded = np.zeros((n, m))
    Z_padded[:s_truncated.shape[0], :s_truncated.shape[0]] = np.diag(s_truncated)

    B = U_padded @ Z_padded @ Vt_padded
    return B